# Evaluating the Effectiveness of Few-Shot Learning in Traffic Sign Recognition Tasks

<h2>ABSTRACT</h2>
<p style="text-align: justify; font-family: Arial; font-size: 15px; line-height: 1.6; font-weight: normal;margin-right: 20px;">Nowadays, self-driving veichles is one of the main application of Artificial Intelligence (AI) which has been most widely used in daily-life scenarios. These veichles are based on many AI models among which there are traffic sign recognition systems. Different proposals have been provided for these systems, most of which rely on the usage of big datasets such as GTSRB (dataset of German traffic signs). Collecting a big amount of data is one of the main problems of Machine Learning models, which need lots of evidences in order to being able to generalize well. The effort in collecting more data than available can be one of the main aspects to consider during the development of these systems. Furthermore, using a big dataset expose to possible Adversarial attacks, from poisoning to evasion ones, specially in case of out-source training and testing. For these reasons, in this work we investigate the application of state-of-art Few-shot learning models on the traffic sign recognition domain. The aim of this work is to define the effectiveness of these approaches, in contrast to non-few-shot ones, to open the possibilities of build more robust and effortless solutions in this field.</p>

# NOTE
più peso alla motivazione dell'attacco rispetto alla difficoltà nella raccolta di dati. Quindi focalizzarsi sul problema che attacchi di poisoning necessitano di aggiungere esempi poisonati che sono facilmente scopribili in casi di dataset piccoli (come nel few-shot) mediante un controllo umano dei sample. Attenzione: verificare, in caso questo controllo non si possa fare, se l'attacco è avvantaggiato con un dataset più piccolo rispetto ad uno grande. Se cosi fosse, sarebbe un problema.

in the related work there are the works we are trying to improve

specificare come ho creato il dataset ridotto

testare i modelli anche sul dataset ridotto

cercare come vengono valutati i modelli di few shot. Se si usa bootstrap o altre tecniche

usare più modelli few shot, non solo CLIP e LDC

-  N-way-k-shot (it means N classes and K instances per classes)

-  Approfondire discorso support set (set non utilizzato nè durante training nè durante test ma solo durante fase di predizione)


# RICERCHE ESISTENTI
- <b>FUSED-Net: Enhancing Few-Shot Traffic Sign Detection with Unfrozen Parameters, Pseudo-Support Sets, Embedding Normalization, and Domain Adaptation</b>: https://arxiv.org/html/2409.14852v1 SOLO SU DATASET DEL BANGLADESH

- <b>Cross-domain Few-shot In-context Learning for Enhancing Traffic Sign Recognition</b>: questo fa un compare tra modelli few shot sul dataset GTSRB

- <b>Few-shot traffic sign recognition with clustering inductive bias and random neural network</b>: questo propone un nuovo modello per few-shot su traffic sign recognition

- <b>Meta-YOLO: Meta-Learning for Few-Shot Traffic Sign Detection via Decoupling Dependencies</b>: questo fa traffic sign detection e non recognition (usa YOLO) quindi a noi non ci interessa

- <b>Research on a Traffic Sign Recognition Method under Small Sample Conditions</b>: nuovo modello proposto

-  <b>Self-supervised few-shot learning for real-time traffic sign classification</b>: altro modello introdotto

<h2>No few-shot models used for the comparison</h2>
<div style="text-align: justify; font-family: Arial; font-size: 15px; line-height: 1.6; font-weight: normal;margin-right: 20px;">
<ul>
<li>An effective automatic traffic sign classification and recognition deep convolutional networks (2022) (Quello di Mattia)
<a href="https://doi.org/10.1007/s11042-022-12531-w">Link Paper</a>
</li>
<li>Multi-column deep neural network for traffic sign classification (2012) (Modello MCDNN)
<a href="https://doi.org/10.1016/j.neunet.2012.02.023">Link Paper</a>
<br><a href="https://github.com/sixftninja/german-traffic-sign-detection/tree/master">Link GitHub</a>
</li>
<li>A Low-cost and Ultra-lightweight Binary Neural Network for Traffic Signal Recognition (2025) (Modello BNN) (No GitHub)
<a href="http://dx.doi.org/10.48550/arXiv.2501.07808">Link Paper</a>
</li>
<li>Visual Trasformer (No Paper, solo codice)
<a href="https://huggingface.co/kelvinandreas/vit-traffic-sign-GTSRB">Link HuggingFace</a>
</li>
</ul>
</div>

<h2>Few-shot models used for the comparison</h2>
<div style="text-align: justify; font-family: Arial; font-size: 15px; line-height: 1.6; font-weight: normal;margin-right: 20px;">
<ul>
<li>(Quello nostro di Computer Vision) Clip encoder (<a href="https://doi.org/10.48550/arXiv.2103.00020">Link Paper</a>) + LCD to calculate the noise (<a href="https://doi.org/10.48550/arXiv.2504.12104">Link Paper</a>)
</li>
</ul>
</div>




# EVALUATION METRICS

Possible evaluation metrics used in the papers above are:
Ma precision e recall non sono per binary classification? Lo dovresti fare per ogni classe considerando tutte le altre classi come istanze negative?
- Average precision (AP): area under the precision-recall curve
- Precision
- Recall
- A sto punto anche F1 Score? Male non fa
- Accuracy rate (consigliato al posto di error rate. più utilizzato in letteratura)
-

# EVALUATION TECHNIQUES FOR FEW SHOTS ALGORITHMS
- <b>Bootstrap</b>
- <b>Fine-Grained Image Classification</b>: si valutano e testano gruppi di classi alla volta in modo tale da identificare i gruppi più problematici (nel nostro caso il gruppo dei segnali stradali relativi ai limiti di velocità potrebbero essere piu difficili da classificare rispetto a quelli di pericolo). "Fine-Grained Image Classification is a task in computer vision where the goal is to classify images into subcategories within a larger category. For example, classifying different species of birds or different types of flowers. This task is considered to be fine-grained because it requires the model to distinguish between subtle differences in visual appearance and patterns, making it more challenging than regular image classification tasks."(https://paperswithcode.com/task/fine-grained-image-classification)
-<b>Generalization error</b>: How much does accuracy drop when we reduce K (e.g., from K=5 to K=1)?*
- <b>Paper on few-shot evaluation techniques</b> https://arxiv.org/pdf/2307.02732

# EVALUATION TECHNIQUES TO COMPARE THE FEW SHOT VS MANY-SHOT LEARNING
- <b>Paired T-test</b>: Usiamo J dataset differenti per il traffic sign recgnition task (uno è quello tedesco, dobbiamo trovarne altri) e testiamo per ciascuno di essi il modello few-shot ed uno no-few shot. Per ottenere l'error rate per ciascun dataset possiamore usare l'holdout o cross-validation (meglio ques'ultimo perchè citato nel paper come migliore metodo di valutazione per modelli few-shot).
1) Usiamo l'holdout assicurandoci di avere stesso identico test set per entrambi i modelli ma training set diverso (per il modello few-shot ovviamente abbiamo bisogno di meno sample rispetto a quello no few-shot).
2) In alternativa all'holdout, possiamo usare una k-fold cross validation in questo modo: il numero e la suddivisione dei fold sono gli stessi per entrambi i modelli, tuttavia, ad ogni iterazione della cross validation, usiamo lo stesso test set per entrambi e, per il modello few-shot, anzichè usare tutte i rimanenti fold per il training set ne prendiamo solo una parte. Dato che il numero di sample per classe nel modello few-shot segue la potenza del 2(Y=2,4,8,16 tipicamente), allora dobbiamo fare in modo che ogni fold abbia Y*N(dove N è il numero delle label e Y il numero dei sample per label richiesto dal modello few-shot) con l'accortezza di averne almeno x=Y/2 che siano strutturati in modo tale da avere SOLTANTO Y istanze per ciascuna delle N label. Problema: cosi abbiamo pochi sample nel test set dato che i fold sono piccoli per venire incontro alle esigenze del modello few-shot (J*N sample in ciascun fold)...Soluzione: anzichè prendere un fold alla volta per il test set ne prendiamo tipo 3 (ovviamente ciò vale per entrambi i modelli). Quindi per il no few-shot prendiamo X fold per il test set e utilizziamo i restanti per il training set, mentre per il few-shot prendiamo sempre X fold per il test set e utilizziamo soltanto quelli che ci servono il training. 


- <b>Intervalli di confidenza</b> A parte dal t-test, per ogni confronto tra due algoritmi ci calcoliamo per entrambe il corrispondente intervallo di confidenza.
- <b>Focus su few-shot</b> A parte dal t-test, testiamo anche il modello few-shot sui diversi gruppi di classi (fine-grained few shot) (no train, solo test) per identificare su quali gruppi il modello few-shot si comporta peggio.





# Nuova sezione

<h2>Code used to create the random dataset:</h2>
import os
import random
import shutil
from PIL import Image

def copy_random_images(p1, p2, num_images_per_folder=5):
    """
    Copies random images from subfolders in p1 to p2, recreating the subfolder structure.

    Args:
        p1 (str): Source path containing subfolders with images.
        p2 (str): Destination path where the subfolder structure and images will be recreated.
        num_images_per_folder (int): Number of random images to copy from each subfolder.
    """
    if not os.path.exists(p1):
        raise ValueError(f"Source path '{p1}' does not exist.")
    
    if not os.path.exists(p2):
        os.makedirs(p2)

    # Iterate through subfolders in p1
    for subfolder in os.listdir(p1):
        subfolder_path = os.path.join(p1, subfolder)
        if os.path.isdir(subfolder_path):
            # Create the same subfolder in p2
            dest_subfolder_path = os.path.join(p2, subfolder)
            os.makedirs(dest_subfolder_path, exist_ok=True)

            # Get all image files in the current subfolder
            image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
            
            # Select random images
            random_images = random.sample(image_files, min(num_images_per_folder, len(image_files)))

            # Copy selected images to the destination subfolder
            for image in random_images:
                src_image_path = os.path.join(subfolder_path, image)
                dest_image_path = os.path.join(dest_subfolder_path, image)

                # Open, resize, and save the image
                with Image.open(src_image_path) as img:
                    img = img.convert("RGB")  # Ensure the image is in RGB format
                    img = img.resize((32, 32))  # Resize to 32x32
                    img.save(dest_image_path)

            print(f"Copied {len(random_images)} images from '{subfolder_path}' to '{dest_subfolder_path}'.")

p1 = "ortiginal_images_folder"
p2 = "destination_images_folder"
num_images_per_folder = 30

arr=["/train","/val","/test"]

for a in arr:
  copy_random_images(p1, p2, num_images_per_folder+a)

# Setup environment

<h2> LOAD THE DATASETS </h2>

In [57]:
from datasets import load_dataset, DatasetDict, Dataset, Image
import pandas as pd
import os

def load_split_images(split_folder):
    image_paths = []
    labels = []
    
    for label in os.listdir(split_folder):
        label_folder = os.path.join(split_folder, label)
        if os.path.isdir(label_folder):
            for file in os.listdir(label_folder):
                if file.lower().endswith(('.ppm')):
                    image_paths.append(os.path.join(label_folder, file))
                    labels.append(label)
    
    return {"image": image_paths, "label": labels}

# Load Belgian Traffic Sign Dataset
belgian_dataset_train = load_split_images(os.path.join("./dataset/datasets/belgium-traffic-sign", "train"))
belgian_dataset_test = load_split_images(os.path.join("./dataset/datasets/belgium-traffic-sign", "test"))

belgian_dataset = DatasetDict({
    "train": Dataset.from_dict(belgian_dataset_train).cast_column("image", Image()),
    "test": Dataset.from_dict(belgian_dataset_test).cast_column("image", Image())
})

# Load datasets from Hugging Face
chinese_dataset = load_dataset("kuchidareo/chinese_trafficsign_dataset", cache_dir="./__hugging_face_cache__")
gtsrb_dataset = load_dataset("bazyl/GTSRB", cache_dir="./__hugging_face_cache__")
russian_dataset = load_dataset("eleldar/rtsd_cleaned", cache_dir="./__hugging_face_cache__")

# Create list of the datasets with a list of the column of the label and of the image
datasets = [chinese_dataset,gtsrb_dataset,russian_dataset,belgian_dataset]
dataset_label_column_name = ["label", "ClassId", "sign_class","label"]
dataset_image_column_name = ["image", "path", "image", "image"]

Found cached dataset parquet (c:/Users/utente/Desktop/uni/anno 1 magistrale/semestre 2/Machine Learning/few-shot-traffic-sign/__hugging_face_cache__/kuchidareo___parquet/kuchidareo--chinese_trafficsign_dataset-29e1f3564d53b6e9/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 1/1 [00:00<00:00, 27.33it/s]
Found cached dataset parquet (c:/Users/utente/Desktop/uni/anno 1 magistrale/semestre 2/Machine Learning/few-shot-traffic-sign/__hugging_face_cache__/bazyl___parquet/bazyl--GTSRB-fb44a5bda34b3b37/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 2/2 [00:00<00:00, 34.83it/s]
Found cached dataset parquet (c:/Users/utente/Desktop/uni/anno 1 magistrale/semestre 2/Machine Learning/few-shot-traffic-sign/__hugging_face_cache__/eleldar___parquet/eleldar--rtsd_cleaned-50c620fbb79ae8f4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 1/1 [00:00<00:00, 30.97it/s]


In [ ]:
# dataset loader command
# torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)

datasets_filtered=[]

# Filter the samples in the datasets, removing labels with less than 20 samples
for i,d in enumerate(datasets):
    print("Dataset: ", i)
    if "test" in d.keys():
        df = pd.concat([d["train"].to_pandas(), d["test"].to_pandas()])
    else:
        df = d["train"].to_pandas()
    
    # Get label frequencies (number of samples for each label)
    label_column = dataset_label_column_name[datasets.index(d)]
    label_frequencies = [(i,df[df[label_column]==i].shape[0]) for i in df[label_column].unique()]
    
    # Removing labels with less than 20 samples
    df_filtered = df[df[label_column].isin([i[0] for i in label_frequencies if i[1] >= 20])]

    datasets_filtered.append(df_filtered)

    print("Number of filtered labels: ", df_filtered[label_column].nunique())
    print("Number of removed labels: ",df[label_column].nunique()-df_filtered[label_column].nunique())
    print("Number of samples in the dataset: ",df_filtered.shape[0])

Dataset:  0
Number of filtered labels:  47
Number of removed labels:  11
Number of samples in the dataset:  5902
Dataset:  1
Number of filtered labels:  43
Number of removed labels:  0
Number of samples in the dataset:  51839
Dataset:  2
Number of filtered labels:  139
Number of removed labels:  59
Number of samples in the dataset:  103841
Dataset:  3
Number of filtered labels:  52
Number of removed labels:  10
Number of samples in the dataset:  6970


<h2>Few-shot models

In [ ]:
from ldc_train import *

dataset_name="traffic-signs"
shots=16
save_dir = dataset_name+"_100_way_"+str(shots)+"_shot"
log_txt_path = Tools.new_dir(os.path.join(LOG_ROOT, save_dir, "results.txt"))

# Create the configuration for the custom dataset
config = Config10Dataset(
    dataset_name=dataset_name,
    seed=2024,
    shots=shots,
    backbone="ViT-B/16",
    lr=0.001,
    batch_size=64,
    train_epoch=50,
    loss_lambda=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],   #last is regularization lambda
    fuse_type=2,
    save_dir=save_dir,
    #regularization='elastic_net'
)

# Initialize the Runner with the custom dataset configuration
runner = Runner(config=config)

# Start the training process
Tools.print("Starting training with the custom dataset...")
acc_list = runner.train()
Tools.print({"name": "traffic-sign", "acc": acc_list, "detail": config.get_detail()}, log_txt_path)

# Optionally, evaluate the model
Tools.print("Evaluating the model...")
test_acc_list = runner.test()
Tools.print({"name": "traffic-sign", "test_acc": test_acc_list}, log_txt_path)


2025-05-22 00:12:08 Preparing ViT-B/16 model.
2025-05-22 00:12:09 Getting cached textual weights W ...
2025-05-22 00:12:11 Preparing traffic-signs dataset.
2025-05-22 00:12:11 Starting training with the custom dataset...


epoch 0:   0%|          | 0/11 [00:00<?, ?it/s]

<h2>NON-Few-shot models</h2>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, ViTImageProcessor
from sklearn.svm import SVC

num_classes = 43

#############################################
# Modified Neural Network Class
##############################################

class ModifiedNN(nn.Module):
    '''
    Parent class for all neural networks model that we will implement.
    It is a basic nn.Module with a training method which implements the training for neural networks.
    '''
    def __init__(self):
        super().__init__()
    
    def training_loop(self, train_loader, optimizer=optim.Adam, loss_function=nn.BCELoss, epochs=10):
        # Training loop
        for epoch in range(epochs):
            model.train()  # Set the model to training mode
            running_loss = 0.0

            for batch in train_loader:
                X_batch, y_batch = batch

                # Forward pass
                predictions = self.forward(X_batch)
                loss = loss_function(predictions, y_batch)

                # Backward pass
                optimizer.zero_grad()  # Clear previous gradients
                loss.backward()        # Compute gradients
                optimizer.step()       # Update parameters

                running_loss += loss.item()

            # Calculate average training loss
            avg_train_loss = running_loss / len(train_loader)

            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Training Loss: {avg_train_loss:.4f}")
        
        return avg_train_loss

##############################################
# Utility functions
###############################################

def from_data_loader_to_numpy(data_loader):
    '''
    Method that concevts a torch data loader to two numpy arrays: X (samples) and y (labels).
    '''
    X_list = []
    y_list = []

    for batch in data_loader:
        X_batch, y_batch = batch

        X_list.append(X_batch.numpy())
        y_list.append(y_batch.numpy())
    
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    
    return X, y

#############################################
# CNN
#############################################

class CNNModel(ModifiedNN):
    '''
    CNN for traffic sign classification. The input is a 32x32x3 image.
    '''
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # padding=1 per "same"
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 5 * 5, num_classes)

        # Transformation used to adapt an input to the desired size wanted by the model
        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5],std=[0.5, 0.5])  # normalize between 0 and 1
        ])
        
    def forward(self, x):
        # Apply transformations
        x = self.transform(x)

        # Forward pass
        x = F.relu(self.conv1(x))  
        x = F.relu(self.conv2(x))  
        x = self.pool1(x)          
        x = self.dropout1(x)
        x = F.relu(self.conv3(x))  
        x = F.relu(self.conv4(x))  
        x = self.pool2(x)          
        x = self.dropout2(x)
        x = self.flatten(x)       
        x = self.fc(x)             
        return F.softmax(x, dim=1)
    
    def training(self, train_loader, epochs=10, lr=0.001):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        loss = nn.CrossEntropyLoss
        return super().training_loop(train_loader, optimizer, loss, epochs)
    
    def summary(self):
        summary(self, input_size=(3, 32, 32))
        

cnn_model = CNNModel(num_classes=num_classes)
if torch.cuda.is_available():
    cnn_model.cuda()

print("#########################\nCNN\n#########################\n")
print(cnn_model.summary())
print("Model created!")

#############################################
# MDCNN
#############################################

class MCDNN(ModifiedNN):
    '''
    MCDNN for traffic sign classification. The input is a 48x48x3 image.
    '''
    def __init__(self, num_classes):
        super(MCDNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 150, kernel_size=7)
        self.bn1 = nn.BatchNorm2d(150)
        self.conv2 = nn.Conv2d(150, 200, kernel_size=4)
        self.bn2 = nn.BatchNorm2d(200)
        self.conv3 = nn.Conv2d(200, 300, kernel_size=4)
        self.bn3 = nn.BatchNorm2d(300)
        self.fc1 = nn.Linear(300 * 3 * 3, 350)
        self.bn4 = nn.BatchNorm1d(350)
        self.fc2 = nn.Linear(350, num_classes)
        self.pool = nn.MaxPool2d(2)
        self.conv_drop = nn.Dropout2d(p=0.2)
        self.fc_drop = nn.Dropout(p=0.5)
        
        # Transformation used to adapt an input to the desired size wanted by the model
        self.transform = transforms.Compose([
            transforms.Resize((48, 48)),
            transforms.ToTensor(),
            transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
        ])

    def forward(self, x):
        # Apply transformations
        x = self.transform(x)

        # Forward pass
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.conv_drop(self.pool(x))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.conv_drop(self.pool(x))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv_drop(self.pool(x))
        x = x.view(-1, 300 * 3 * 3)
        x = F.relu(self.bn4(self.fc1(x)))
        x = self.fc2(self.fc_drop(x))
        return F.log_softmax(x, dim=1)
    
    def training(self, train_loader, epochs=10, lr=0.001):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        loss = F.nll_loss
        return super().training_loop(train_loader, optimizer, loss, epochs)
    
    def summary(self):
        summary(self, input_size=(3, 48, 48))

mcdnn_model = MCDNN(num_classes=num_classes)
if torch.cuda.is_available():
    mcdnn_model.cuda()

print("\n\n#########################\nMDCNN\n#########################\n")
print(mcdnn_model.summary())
print("Model created!")

#############################################
# KNN
#############################################

class KNNClassifier:
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None

    def forward(self, X):
        # Calculate Euclidean distance
        distances = torch.cdist(X, self.X_train)
        
        # Get indices of k nearest neighbors
        knn_indices = distances.topk(self.k, largest=False).indices
        
        # Retrieve labels of k nearest neighbors
        knn_labels = self.y_train[knn_indices]
        
        # Find most common label in neighbors
        predictions = torch.mode(knn_labels, dim=1).values
        return predictions

    def training(self, train_loader, epochs=10, lr=0.001):
        X, y = from_data_loader_to_numpy(train_loader)

        # Store training data for distance calculations
        self.X_train = X
        self.y_train = y

knn = KNNClassifier(k=5)
    
print("\n\n#########################\nKNN\n#########################\n")
print("Model created!")

#############################################
# SVM
#############################################

class SVM():
    def __init__(self):
        self.model = SVC(kernel='linear')

    def forward(self, X):
        return self.model.predict(X)

    def train(self, train_loader, epochs=10, lr=0.001):
        X, y = from_data_loader_to_numpy(train_loader)

        self.model.fit(X,y, lr=lr)

print("\n\n#########################\nSVM\n#########################\n")
print("Model created!")

#############################################
# ViT
#############################################

processor = ViTImageProcessor.from_pretrained("kelvinandreas/vit-traffic-sign-GTSRB")
model = ViTForImageClassification.from_pretrained("kelvinandreas/vit-traffic-sign-GTSRB")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]           2,432
            Conv2d-2           [-1, 32, 26, 26]           9,248
         MaxPool2d-3           [-1, 32, 13, 13]               0
           Dropout-4           [-1, 32, 13, 13]               0
            Conv2d-5           [-1, 64, 13, 13]          18,496
            Conv2d-6           [-1, 64, 11, 11]          36,928
         MaxPool2d-7             [-1, 64, 5, 5]               0
           Dropout-8             [-1, 64, 5, 5]               0
           Flatten-9                 [-1, 1600]               0
           Linear-10                   [-1, 43]          68,843
Total params: 135,947
Trainable params: 135,947
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.62
Params size (MB): 0.52
Estimated T